# Pour le polluant PM10

### On importe les bibliothèques

In [32]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Conversion des données en DataFrame

In [33]:
# URL et paramètres de la requête
url = "https://data.airpl.org/api/v1/mesure/mensuelle/"
params = {
    "code_configuration_de_mesure__code_point_de_prelevement__code_polluant": 24,
    "date_heure_tu__range": "2021-1-1,2023-12-31",
    "export": "json"
}

# Récupérer les données
response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    dfPM10 = pd.DataFrame(data['results'])  # Adapter selon la structure des données JSON

    # Sauvegarder le DataFrame pour utilisation ultérieure
    dfPM10.to_pickle('PM10.pkl')
else:
    print(f"Erreur {response.status_code}: {response.text}")

# Afficher les premières lignes du DataFrame
print(dfPM10.head())


                                  id code_polluant code_point_de_prelevement  \
0  2023-12-01 00:00:00FR23003_PM10_1            24              FR23003_PM10   
1  2023-12-01 00:00:00FR23068_PM10_4            24              FR23068_PM10   
2  2023-12-01 00:00:00FR23070_PM10_3            24              FR23070_PM10   
3  2023-12-01 00:00:00FR23078_PM10_2            24              FR23078_PM10   
4  2023-12-01 00:00:00FR23107_PM10_4            24              FR23107_PM10   

  code_station               nom_station               nom_commune  \
0      FR23003              LA MEGRETAIS                    Donges   
1      FR23068                   FROSSAY                   Frossay   
2      FR23070  SAINT ETIENNE DE MONTLUC  Saint-Etienne-De-Montluc   
3      FR23078             SAINT EXUPERY                    Cholet   
4      FR23107            LA CHAUVINIERE                    Nantes   

  code_commune departement_code   departement_nom  code_zone_affichage  \
0        44052          

In [34]:
# Afficher les valeurs uniques de la colonne 'departement_nom'
valeurs_uniques = dfPM10['departement_nom'].unique()
print("Valeurs uniques de la colonne 'departement_nom' :")
print(valeurs_uniques)


Valeurs uniques de la colonne 'departement_nom' :
['Loire-Atlantique' 'Maine-et-Loire' 'Mayenne' 'Vendée' 'Sarthe']


In [35]:
# Afficher le nombre de valeurs uniques dans la colonne 'departement_nom'
nombre_valeurs_uniques = dfPM10['departement_nom'].nunique()
print("Nombre de valeurs uniques dans la colonne 'departement_nom' :")
print(nombre_valeurs_uniques)


Nombre de valeurs uniques dans la colonne 'departement_nom' :
5


In [36]:
dfPM10.info

<bound method DataFrame.info of                                     id code_polluant  \
0    2023-12-01 00:00:00FR23003_PM10_1            24   
1    2023-12-01 00:00:00FR23068_PM10_4            24   
2    2023-12-01 00:00:00FR23070_PM10_3            24   
3    2023-12-01 00:00:00FR23078_PM10_2            24   
4    2023-12-01 00:00:00FR23107_PM10_4            24   
..                                 ...           ...   
724  2021-01-01 00:00:00FR23188_PM10_2            24   
725  2021-01-01 00:00:00FR23188_PM10_4            24   
726  2021-01-01 00:00:00FR23238_PM10_A            24   
727  2021-01-01 00:00:00FR23239_PM10_A            24   
728  2021-01-01 00:00:00FR23249_PM10_1            24   

    code_point_de_prelevement code_station               nom_station  \
0                FR23003_PM10      FR23003              LA MEGRETAIS   
1                FR23068_PM10      FR23068                   FROSSAY   
2                FR23070_PM10      FR23070  SAINT ETIENNE DE MONTLUC   
3      

In [37]:

base_url = "https://data.paysdelaloire.fr/api/explore/v2.1/catalog/datasets/12002701600563_population_pays_de_la_loire_2019_communes_epci/records"
limit = 100  # Nombre de résultats par page
offset = 0  # Offset initial
dfPopulation =  pd.DataFrame()  # Liste pour stocker tous les enregistrements

while True:
    # Construire l'URL avec le paramètre d'offset
    url = f"{base_url}?limit={limit}&offset={offset}"

#Appeler l'API
    response = requests.get(url)

#Vérifier si la requête a réussi
    if response.status_code != 200:
        print(f"Erreur lors de l'appel de l'API: {response.status_code}")
        break

    data = response.json()
#Récupérer les enregistrements
    results = data.get('results', [])
    if results:
        df_results = pd.json_normalize(results)
        dfPopulation = pd.concat([dfPopulation, df_results], ignore_index=True)

#Vérifier si le nombre d'enregistrements récupérés est inférieur au limite
    if len(results) < limit:
        break  # Arrêter la boucle si tous les enregistrements ont été récupérés

#Mettre à jour l'offset pour la prochaine itération
    offset += limit

print(f"Total records retrieved: {len(dfPopulation)}")

Total records retrieved: 1238


In [38]:
dfPopulation.info

<bound method DataFrame.info of       code_region  nom_de_la_region code_departement code_arrondissement  \
0              52  Pays de la Loire               44                   3   
1              52  Pays de la Loire               44                   2   
2              52  Pays de la Loire               44                   3   
3              52  Pays de la Loire               44                   2   
4              52  Pays de la Loire               44                   3   
...           ...               ...              ...                 ...   
1233           52  Pays de la Loire               85                   1   
1234           52  Pays de la Loire               85                   2   
1235           52  Pays de la Loire               85                   1   
1236           52  Pays de la Loire               85                   1   
1237           52  Pays de la Loire               85                   1   

     code_canton code_commune      nom_de_la_commune  p

In [39]:
dfPopulation.isnull().sum()

code_region                          0
nom_de_la_region                     0
code_departement                     0
code_arrondissement                  0
code_canton                          0
code_commune                         0
nom_de_la_commune                    0
population_municipale                0
population_comptee_a_part            0
population_totale                    0
epci                                 9
departement                          0
tranche_population                   0
geo_shape.type                       8
geo_shape.geometry.coordinates       8
geo_shape.geometry.type              8
geo_point_2d.lon                     8
geo_point_2d.lat                     8
geo_shape                         1238
geo_point_2d                      1238
dtype: int64

Observations : 

On remarque ici que les colonnes geo_shape et geo_point_2d sont vides. En effet ces deux colonnes ont des sous ensembles qui sont récuperées en étant préfixé par le nom de ces colonnes. On peut donc supprimer ces 2 colonnes qui sont entièrement vides.
On remarque également que quelques colonnes sont vides sur une dizaine de lignes, dependant sur un total de 1238 lignes ce n'est pas significatif, on garde donc ces lignes ici.


In [40]:
# Afficher les premières lignes de la colonne 'departement'
print("Premières lignes de la colonne 'departement' :")
print(dfPopulation['departement'].head(200))

Premières lignes de la colonne 'departement' :
0      Loire-Atlantique
1      Loire-Atlantique
2      Loire-Atlantique
3      Loire-Atlantique
4      Loire-Atlantique
             ...       
195             Mayenne
196             Mayenne
197             Mayenne
198             Mayenne
199             Mayenne
Name: departement, Length: 200, dtype: object


### Suppression des colonnes geo_shape et geo_point_2d

In [41]:
# Suppression des colonnes 'geo_shape' et 'geo_point_2d' de dfPopulation
dfPopulation = dfPopulation.drop(columns=['geo_shape', 'geo_point_2d'], errors='ignore')

# Afficher les premières lignes du DataFrame dfPopulation pour vérifier les modifications
print(dfPopulation.head())


   code_region  nom_de_la_region code_departement code_arrondissement  \
0           52  Pays de la Loire               44                   3   
1           52  Pays de la Loire               44                   2   
2           52  Pays de la Loire               44                   3   
3           52  Pays de la Loire               44                   2   
4           52  Pays de la Loire               44                   3   

  code_canton code_commune nom_de_la_commune  population_municipale  \
0          09        44006           Assérac                 1790.0   
1          21        44018            Bouaye                 7844.0   
2          89        44005   Chaumes-en-Retz                 6759.0   
3          24        44047           Couëron                21372.0   
4          02        44049        Le Croisic                 4093.0   

   population_comptee_a_part  population_totale  \
0                       32.0             1822.0   
1                      153.0    

In [42]:
# Fusionner les deux DataFrames en utilisant les colonnes 'nom_commune' et 'nom_de_la_commune' avec une jointure externe gauche
if 'nom_commune' in dfPM10.columns and 'nom_de_la_commune' in dfPopulation.columns:
    dfMerged = pd.merge(dfPM10, dfPopulation, left_on='nom_commune', right_on='nom_de_la_commune', how='left', suffixes=('_pm10', '_pop'))
    
    # Sauvegarder le DataFrame fusionné pour utilisation ultérieure
    dfMerged.to_pickle('MergedData.pkl')

    # Afficher les premières lignes du DataFrame fusionné
    print("dfMerged:")
    print(dfMerged.head())
else:
    print("Les colonnes de fusion 'nom_commune' et 'nom_de_la_commune' ne sont pas présentes dans les DataFrames. Vérifiez les colonnes disponibles.")


dfMerged:
                                  id code_polluant code_point_de_prelevement  \
0  2023-12-01 00:00:00FR23003_PM10_1            24              FR23003_PM10   
1  2023-12-01 00:00:00FR23068_PM10_4            24              FR23068_PM10   
2  2023-12-01 00:00:00FR23070_PM10_3            24              FR23070_PM10   
3  2023-12-01 00:00:00FR23078_PM10_2            24              FR23078_PM10   
4  2023-12-01 00:00:00FR23107_PM10_4            24              FR23107_PM10   

  code_station               nom_station               nom_commune  \
0      FR23003              LA MEGRETAIS                    Donges   
1      FR23068                   FROSSAY                   Frossay   
2      FR23070  SAINT ETIENNE DE MONTLUC  Saint-Etienne-De-Montluc   
3      FR23078             SAINT EXUPERY                    Cholet   
4      FR23107            LA CHAUVINIERE                    Nantes   

  code_commune_pm10 departement_code   departement_nom  code_zone_affichage  \
0        

In [43]:
# Afficher les premières lignes de la colonne 'departement'
print("Premières lignes de la colonne 'departement' :")
print(dfMerged['departement'].head(200))


Premières lignes de la colonne 'departement' :
0      Loire-Atlantique
1      Loire-Atlantique
2                   NaN
3        Maine-et-Loire
4      Loire-Atlantique
             ...       
195      Maine-et-Loire
196    Loire-Atlantique
197    Loire-Atlantique
198      Maine-et-Loire
199             Mayenne
Name: departement, Length: 200, dtype: object


In [44]:
# Charger le DataFrame depuis le fichier sauvegardé
dfPM10 = pd.read_pickle('MergedData.pkl')

# Afficher les types de chaque colonne
print(dfPM10.dtypes)


id                                 object
code_polluant                      object
code_point_de_prelevement          object
code_station                       object
nom_station                        object
nom_commune                        object
code_commune_pm10                  object
departement_code                   object
departement_nom                    object
code_zone_affichage                 int64
date_heure_tu                      object
date_heure_local                   object
valeur_originale                  float64
valeur                            float64
validite                             bool
code_configuration_de_mesure       object
code_region                       float64
nom_de_la_region                   object
code_departement                   object
code_arrondissement                object
code_canton                        object
code_commune_pop                   object
nom_de_la_commune                  object
population_municipale             

## Analyse et correction du dataset

### Types et valeurs manquantes par variable

In [45]:
dfPM10.shape

(729, 34)

In [46]:
dfPM10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729 entries, 0 to 728
Data columns (total 34 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              729 non-null    object 
 1   code_polluant                   729 non-null    object 
 2   code_point_de_prelevement       729 non-null    object 
 3   code_station                    729 non-null    object 
 4   nom_station                     729 non-null    object 
 5   nom_commune                     729 non-null    object 
 6   code_commune_pm10               729 non-null    object 
 7   departement_code                729 non-null    object 
 8   departement_nom                 729 non-null    object 
 9   code_zone_affichage             729 non-null    int64  
 10  date_heure_tu                   729 non-null    object 
 11  date_heure_local                729 non-null    object 
 12  valeur_originale                684 

In [47]:
dfPM10.isnull().sum()

id                                  0
code_polluant                       0
code_point_de_prelevement           0
code_station                        0
nom_station                         0
nom_commune                         0
code_commune_pm10                   0
departement_code                    0
departement_nom                     0
code_zone_affichage                 0
date_heure_tu                       0
date_heure_local                    0
valeur_originale                   45
valeur                             45
validite                            0
code_configuration_de_mesure        0
code_region                       172
nom_de_la_region                  172
code_departement                  172
code_arrondissement               172
code_canton                       172
code_commune_pop                  172
nom_de_la_commune                 172
population_municipale             172
population_comptee_a_part         172
population_totale                 172
epci        

Observations : 
- On remarque que pour les colonnes "valeur_originale" et "valeur" on a quelques lignes vides (45), mais aucune colonne entièrement vide
- On à également 172 lignes vides que l'on retrouves sur beaucoup de colonnes, il faudra surrement les supprimés après une vérification de ce qu'elles contiennent
- Les types des différentes colonnes semblent cohérant par rapport à ce qu'elles affichent

## Visualiser les différentes données de chaque colonne

### Pour les colonnes de type float64

In [48]:
for col in dfPM10.select_dtypes('float64'):
    print(f'{col :-<20} {dfPM10[col].unique()}')

valeur_originale---- [14.703268 13.845743 13.207121 14.437269 14.799597 16.016296 13.844456
 13.69738  11.212879 11.491947 13.644631 11.693145 12.503688 11.722379
 15.213663 19.120575 17.169493 14.915053 15.857918 17.315884 15.804879
 13.688317       nan 12.89322  16.59757  15.26243  15.72821  13.129271
 13.82365  12.325356 12.232453 14.993402 11.575278 12.933588 11.598619
 15.479178 18.541807 15.948601 13.80017  15.93206  18.507591 12.995968
 11.939107 11.952682 15.144767 13.050617 14.15175  15.055115 15.018212
 13.518208 14.214963 14.134863 12.936828 11.582056 13.323688 14.617453
 17.685814 13.622661 12.886518 13.599592 15.197661 16.376551 14.728593
 15.241199 15.466693 15.189548 15.572867 16.44437  17.205671 13.878689
 15.086312 14.49725  14.281131 15.783613 14.516237 15.575665 13.882277
 16.398445 19.339804 12.570506 15.912861 16.470591 15.935651 10.124685
  9.114499  8.448091 10.688625  8.433603  8.48069  10.013508  9.44375
  9.47298  10.543723 10.132714 10.020132  9.217624  8.931

In [49]:
for col in dfPM10.select_dtypes('int64'):
    print(f'{col :-<20} {dfPM10[col].unique()}')

code_zone_affichage- [ 3  4  1  2  5  6 13  7 11  8]


In [50]:
# Convertir les colonnes contenant des dictionnaires en chaînes de caractères
dfPM10 = dfPM10.applymap(lambda x: str(x) if isinstance(x, dict) else x)

# Afficher les valeurs uniques des colonnes de type object
for col in dfPM10.select_dtypes(include=['object']).columns:
    print(f'{col :-<20} {dfPM10[col].unique()}')


id------------------ ['2023-12-01 00:00:00FR23003_PM10_1' '2023-12-01 00:00:00FR23068_PM10_4'
 '2023-12-01 00:00:00FR23070_PM10_3' '2023-12-01 00:00:00FR23078_PM10_2'
 '2023-12-01 00:00:00FR23107_PM10_4' '2023-12-01 00:00:00FR23110_PM10_3'
 '2023-12-01 00:00:00FR23120_PM10_4' '2023-12-01 00:00:00FR23123_PM10_2'
 '2023-12-01 00:00:00FR23124_PM10_3' '2023-12-01 00:00:00FR23124_PM10_4'
 '2023-12-01 00:00:00FR23152_PM10_4' '2023-12-01 00:00:00FR23177_PM10_4'
 '2023-12-01 00:00:00FR23178_PM10_3' '2023-12-01 00:00:00FR23182_PM10_3'
 '2023-12-01 00:00:00FR23188_PM10_4' '2023-12-01 00:00:00FR23238_PM10_A'
 '2023-12-01 00:00:00FR23239_PM10_A' '2023-12-01 00:00:00FR23242_PM10_1'
 '2023-12-01 00:00:00FR23249_PM10_1' '2023-12-01 00:00:00FR23251_PM10_1'
 '2023-11-01 00:00:00FR23003_PM10_1' '2023-11-01 00:00:00FR23068_PM10_4'
 '2023-11-01 00:00:00FR23070_PM10_2' '2023-11-01 00:00:00FR23070_PM10_3'
 '2023-11-01 00:00:00FR23078_PM10_2' '2023-11-01 00:00:00FR23107_PM10_4'
 '2023-11-01 00:00:00FR23110_P

/var/folders/1c/70ngsvd53b5fbc_qfg773ckc0000gn/T/ipykernel_5389/2406762181.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfPM10 = dfPM10.applymap(lambda x: str(x) if isinstance(x, dict) else x)


TypeError: unhashable type: 'list'

Observations : 
Pour les colonnes de types float64, il a également la présence de NaN : 
- valeur_originale
- valeur
- code_region
- population_municipale
- population_comptee_a_part
- population_totale
- geo_point_2d.lon
- geo_point_2d.lat

Nous allons donc supprimer les lignes ou ces dernières sont présentes, et donc supprimer toutes les valeurs qui seront négatives pour prévoir si jamais ces valeurs seraient rentrées par erreurs

Pour les colonnes de types objects, il a également la présence de NaN : 
- nom_de_la_region
- code_departement
- code_arrondissement
- code_canton
- code_commune_pop
- departement
- tranche_population
- geo_shape.type


On retrouve ici la présence des 172 lignes vides qu'on va donc supprimées.

### Supression des valeurs "nan" et "-999" pour les colonnes de types float64

In [ ]:
# Liste des colonnes à traiter
colonnes_a_traiter = [
    'valeur_originale', 'valeur', 'code_region', 'population_municipale',
    'population_comptee_a_part', 'population_totale', 'geo_point_2d.lon', 'geo_point_2d.lat'
]

# Supprimer les lignes comportant des valeurs "NaN" dans les colonnes spécifiées
dfPM10.dropna(subset=colonnes_a_traiter, inplace=True)

# Supprimer les lignes comportant des valeurs négatives dans les colonnes spécifiées
dfPM10 = dfPM10[(dfPM10['valeur'] >= 0) & (dfPM10['valeur_originale'] >= 0) &
                (dfPM10['population_municipale'] >= 0) & (dfPM10['population_comptee_a_part'] >= 0) &
                (dfPM10['population_totale'] >= 0) & (dfPM10['geo_point_2d.lon'] >= 0) & 
                (dfPM10['geo_point_2d.lat'] >= 0)]


In [ ]:
for col in dfPM10.select_dtypes('float64'):
    print(f'{col :-<20} {dfPM10[col].unique()}')

### Supression des valeurs "nan" et "-999" pour les colonnes de types objects

In [ ]:
# Liste des colonnes de type object à traiter
colonnes_objects = [
    'nom_de_la_region', 'code_departement', 'code_arrondissement', 
    'code_canton', 'code_commune_pop', 'departement', 
    'tranche_population', 'geo_shape.type'
]
# Supprimer les lignes comportant des chaînes vides dans les colonnes de type object
for col in colonnes_objects:
    dfPM10 = dfPM10[dfPM10[col].str.strip() != '']


In [ ]:
# Afficher les valeurs uniques des colonnes de type object
for col in dfPM10.select_dtypes(include=['object']).columns:
    print(f'{col :-<20} {dfPM10[col].unique()}')

## Comparaison avec les valeurs de seuils

Nous faisons le choix dans un premier temps de ne traiter que les données mensuelles, en prenant en granularité les données mensuelles, de manière à comparer les trimestres des différentes années

In [ ]:
# Définition des seuils pour les particules PM10 en µg/m3
OBJECTIF_QUALITE_PM10 = 30  # en moyenne annuelle
VALEUR_LIMITE_ANNUELLE_PM10 = 40  # en moyenne annuelle

# Afficher les seuils définis
print(f"Objectif de qualité PM10 : {OBJECTIF_QUALITE_PM10} µg/m3 (en moyenne annuelle)")
print(f"Valeur limite annuelle PM10 : {VALEUR_LIMITE_ANNUELLE_PM10} µg/m3 (en moyenne annuelle)")


Création d'une variable cible qui vaut 1 si la valeur est supérieure au seuil et 0 sinon
On ajoute ces colonnes au dataframe

In [ ]:
# Convertir les colonnes de date en format datetime
dfPM10['date_heure_tu'] = pd.to_datetime(dfPM10['date_heure_tu'])

# Ajouter des colonnes de comparaison avec les seuils (variable cible)
dfPM10['cible_objectif_qualite_PM10'] = (dfPM10['valeur'] > OBJECTIF_QUALITE_PM10).astype(int)
dfPM10['cible_valeur_limite_annuelle_PM10'] = (dfPM10['valeur'] > VALEUR_LIMITE_ANNUELLE_PM10).astype(int)

# Afficher les premières lignes du DataFrame avec les nouvelles colonnes de comparaison
print(dfPM10.head())

# Sauvegarder le DataFrame avec les nouvelles colonnes de comparaison
dfPM10.to_pickle('PM10_with_comparison.pkl')


### Distribution des variables cibles : 

In [ ]:
# Afficher le nombre de valeurs 0 et 1 dans la colonne 'cible_objectif_qualite_PM10'
print(dfPM10['cible_objectif_qualite_PM10'].value_counts())
print(dfPM10['cible_valeur_limite_annuelle_PM10'].value_counts())


Observations : on remarque ici qu'aucune valeur ne dépasse le seuil, nous avons donc fait le choix de modifier ce dernier de manière arbitraire pour rendre l'exercice plus intéressant, toutes les valeurs seront donc divisées par 2
De plus pour visualiser les résultats par mois, on décide de créer une valeur de seuil mensuelle nommé "OBJECTIF_QUALITE_PM10_MENSUEL" et qui aura pour valeur 35 (car la moyenne annuelle étant à 30 et la valeur limite annuelle étant à 40 on prend l'entre deux) qu'on va venir diviser par 2 pour avoir des résultats, on va prendre 18 pour arrondir à la valeur au-dessus

In [ ]:
# Définition des nouveaux seuils pour les particules PM10 en µg/m3
OBJECTIF_QUALITE_PM10 = 15  # en moyenne annuelle
VALEUR_LIMITE_ANNUELLE_PM10 = 20  # en moyenne annuelle
OBJECTIF_QUALITE_PM10_MENSUEL = 18 #en moyenne mensuelle

# Afficher les nouveaux seuils définis
print(f"Objectif de qualité PM10 : {OBJECTIF_QUALITE_PM10} µg/m3 (en moyenne annuelle)")
print(f"Valeur limite annuelle PM10 : {VALEUR_LIMITE_ANNUELLE_PM10} µg/m3 (en moyenne annuelle)")
print(f"Objectif de qualité PM10 mensuelle : {OBJECTIF_QUALITE_PM10_MENSUEL} µg/m3 (en moyenne mensuelle)")

In [ ]:
# Convertir les colonnes de date en format datetime
dfPM10['date_heure_tu'] = pd.to_datetime(dfPM10['date_heure_tu'])

# Ajouter des colonnes de comparaison avec les seuils (variable cible)
dfPM10['cible_objectif_qualite_PM10'] = (dfPM10['valeur'] > OBJECTIF_QUALITE_PM10).astype(int)
dfPM10['cible_valeur_limite_annuelle_PM10'] = (dfPM10['valeur'] > VALEUR_LIMITE_ANNUELLE_PM10).astype(int)
dfPM10['cible_objectif_qualite_PM10_mensuelle'] = (dfPM10['valeur'] > OBJECTIF_QUALITE_PM10_MENSUEL).astype(int)

# Afficher les premières lignes du DataFrame avec les nouvelles colonnes de comparaison
print(dfPM10.head())

# Sauvegarder le DataFrame avec les nouvelles colonnes de comparaison
dfPM10.to_pickle('PM10_with_comparison.pkl')

In [ ]:
# Afficher le nombre de valeurs 0 et 1 dans la colonne 'cible_objectif_qualite_PM10'
print(dfPM10['cible_objectif_qualite_PM10'].value_counts())
print(dfPM10['cible_valeur_limite_annuelle_PM10'].value_counts())
print(dfPM10['cible_objectif_qualite_PM10_mensuelle'].value_counts())


Observations : 
- Les variables cible_objectif_qualite_PM10 et cible_objectif_qualite_PM10_mensuelle sont plutot équilibrés, il y à donc un problème sur l'objectif qualité PM10, avec beaucoup d'alertes
- La variable cible_valeur_limite_annuelle_PM10 est plutot déséquilibré, l'indice semble donc plutôt correcte.

## Visualisation des données par des graphiques

Graphique pour voir par année, et par département quand est-ce qu'on à dépassé le seuil de cible_objectif_qualité_PM10 (valeur de seuil = 15, en rouge quand depassé)

In [ ]:
# Ajouter des colonnes pour les années
dfPM10['date'] = pd.to_datetime(dfPM10['date_heure_tu']).dt.tz_localize(None)
dfPM10['année'] = dfPM10['date'].dt.year

# Calculer la moyenne annuelle de PM10 pour chaque département
annual_means = dfPM10.groupby(['année', 'departement_nom'])['valeur'].mean().reset_index()

# Ajouter une colonne indiquant si la moyenne annuelle dépasse les seuils
annual_means['depasse_seuil'] = (annual_means['valeur'] > OBJECTIF_QUALITE_PM10) | (annual_means['valeur'] > VALEUR_LIMITE_ANNUELLE_PM10)

# Créer la palette de couleurs
annual_means['color'] = ['red' if x else 'blue' for x in annual_means['depasse_seuil']]

# Créer un graphique en bâtons pour chaque année
years = annual_means['année'].unique()

for year in years:
    plt.figure(figsize=(15, 8))
    yearly_data = annual_means[annual_means['année'] == year]
    colors = yearly_data['color']
    plt.bar(yearly_data['departement_nom'], yearly_data['valeur'], color=colors)
    
    # Ajouter des lignes horizontales pour les seuils
    plt.axhline(y=OBJECTIF_QUALITE_PM10, color='green', linestyle='--', linewidth=2, label=f'Seuil {OBJECTIF_QUALITE_PM10}')
    plt.axhline(y=VALEUR_LIMITE_ANNUELLE_PM10, color='orange', linestyle='--', linewidth=2, label=f'Seuil {VALEUR_LIMITE_ANNUELLE_PM10}')
    
    plt.xlabel('Département')
    plt.ylabel('Moyenne annuelle PM10')
    plt.title(f'Moyennes annuelles des valeurs PM10 par département pour l\'année {year}')
    plt.legend(title='Seuils')
    plt.xticks(rotation=90)  # Rotation pour mieux afficher les noms des départements
    plt.show()




Graphique pour voir par année, quand est-ce qu'on à dépassé le seuil de cible_objectif_qualité_PM10 (valeur de seuil = 15, en rouge quand depassé)

In [ ]:
# Grouper les données par année
dfPM10['année'] = dfPM10['date_heure_tu'].dt.year
annual_means = dfPM10.groupby('année')['valeur'].mean().reset_index()

# Créer la colonne indiquant si la moyenne annuelle dépasse le seuil
annual_means['depasse_seuil'] = (annual_means['valeur'] > OBJECTIF_QUALITE_PM10).astype(int)

# Créer le diagramme en bâtons
plt.figure(figsize=(10, 5))
colors = ['red' if x == 1 else 'blue' for x in annual_means['depasse_seuil']]
plt.bar(annual_means['année'], annual_means['valeur'], color=colors)
plt.axhline(y=OBJECTIF_QUALITE_PM10, color='green', linestyle='--', linewidth=2, label=f'Seuil {OBJECTIF_QUALITE_PM10_MENSUEL}')
plt.xlabel('Année')
plt.ylabel('Moyenne annuelle PM10')
plt.title('Moyennes annuelles des valeurs PM10')
plt.xticks(annual_means['année'])
plt.show()


Graphique pour voir par année, quand est-ce qu'on à dépassé le seuil de la valeur limite annuelle (valeur de seuil = 20, en rouge quand depassé)

In [ ]:
# Créer la colonne indiquant si la moyenne annuelle dépasse le seuil
annual_means['depasse_seuil'] = (annual_means['valeur'] > VALEUR_LIMITE_ANNUELLE_PM10).astype(int)

# Créer le diagramme en bâtons
plt.figure(figsize=(10, 5))
colors = ['red' if x == 1 else 'blue' for x in annual_means['depasse_seuil']]
plt.bar(annual_means['année'], annual_means['valeur'], color=colors)
plt.axhline(y=VALEUR_LIMITE_ANNUELLE_PM10, color='green', linestyle='--', linewidth=2, label=f'Seuil {VALEUR_LIMITE_ANNUELLE_PM10}')
plt.xlabel('Année')
plt.ylabel('Moyenne annuelle PM10')
plt.title('Moyennes annuelles des valeurs PM10')
plt.xticks(annual_means['année'])
plt.legend()
plt.show()


#### Fusion des 2 graphiques, si un des seuils est dépassé, la colonne devient rouge

In [ ]:
# Créer la colonne indiquant si la moyenne annuelle dépasse le seuil
annual_means['depasse_seuil'] = ((annual_means['valeur'] > OBJECTIF_QUALITE_PM10).astype(int) |
                                (annual_means['valeur'] > VALEUR_LIMITE_ANNUELLE_PM10).astype(int))

# Créer le diagramme en bâtons
plt.figure(figsize=(10, 5))
colors = ['red' if x == 1 else 'blue' for x in annual_means['depasse_seuil']]
plt.bar(annual_means['année'], annual_means['valeur'], color=colors)
plt.axhline(y=OBJECTIF_QUALITE_PM10, color='green', linestyle='--', linewidth=2, label=f'Seuil {OBJECTIF_QUALITE_PM10_MENSUEL}')
plt.axhline(y=VALEUR_LIMITE_ANNUELLE_PM10, color='orange', linestyle='--', linewidth=2, label=f'Seuil {VALEUR_LIMITE_ANNUELLE_PM10}')
plt.xlabel('Année')
plt.ylabel('Moyenne annuelle PM10')
plt.title('Moyennes annuelles des valeurs PM10')
plt.xticks(annual_means['année'])
plt.legend()
plt.show()

Graphique pour voir trimestriel par département, quand est-ce qu'on à dépassé le seuil de cible_objectif_qualité_PM10_mensuel (valeur de seuil = 18, en rouge quand depassé)

In [51]:
# Calculer la moyenne trimestrielle de PM10 pour chaque département et chaque année
quarterly_means = dfPM10.groupby(['année', 'trimestre', 'departement_nom'])['valeur'].mean().reset_index()

# Ajouter une colonne indiquant si la moyenne trimestrielle dépasse les seuils
quarterly_means['depasse_seuil'] = (quarterly_means['valeur'] > OBJECTIF_QUALITE_PM10) | (quarterly_means['valeur'] > VALEUR_LIMITE_ANNUELLE_PM10)

# Créer la palette de couleurs
quarterly_means['color'] = ['red' if x else 'blue' for x in quarterly_means['depasse_seuil']]

# Remplacer 'Q' par 'T' dans les trimestres pour l'affichage
quarterly_means['trimestre'] = quarterly_means['trimestre'].astype(str).str.replace('Q', 'T')

# Créer un graphique en bâtons pour chaque année
years = quarterly_means['année'].unique()

for year in years:
    plt.figure(figsize=(15, 8))
    annual_data = quarterly_means[quarterly_means['année'] == year]
    
    # Boucler sur les trimestres pour créer les groupes
    for quarter in annual_data['trimestre'].unique():
        quarterly_data = annual_data[annual_data['trimestre'] == quarter]
        colors = quarterly_data['color']
        plt.bar(quarterly_data['departement_nom'] + " " + quarterly_data['trimestre'], quarterly_data['valeur'], color=colors)

    # Ajouter des lignes horizontales pour les seuils
    plt.axhline(y=OBJECTIF_QUALITE_PM10, color='green', linestyle='--', linewidth=2, label=f'Seuil {OBJECTIF_QUALITE_PM10}')
    plt.axhline(y=VALEUR_LIMITE_ANNUELLE_PM10, color='orange', linestyle='--', linewidth=2, label=f'Seuil {VALEUR_LIMITE_ANNUELLE_PM10}')

    plt.xlabel('Département et Trimestre')
    plt.ylabel('Moyenne trimestrielle PM10')
    plt.title(f'Moyennes trimestrielles des valeurs PM10 par département pour l\'année {year}')
    plt.legend(title='Seuils')
    plt.xticks(rotation=90)  # Rotation pour mieux afficher les noms des départements
    plt.show()


KeyError: 'année'

Graphique pour voir par année et par mois, quand est-ce qu'on à dépassé le seuil de cible_objectif_qualité_PM10_mensuel (valeur de seuil = 18, en rouge quand depassé)

In [ ]:
import calendar

# Grouper les données par année et mois
dfPM10['année'] = dfPM10['date_heure_tu'].dt.year
dfPM10['mois'] = dfPM10['date_heure_tu'].dt.month
grouped = dfPM10.groupby(['année', 'mois'])[['valeur']].mean().reset_index()

# Créer la colonne indiquant si la moyenne mensuelle dépasse le seuil
grouped['depasse_seuil'] = (grouped['valeur'] > OBJECTIF_QUALITE_PM10_MENSUEL).astype(int)

# Créer le diagramme en bâtons pour chaque année
for year in grouped['année'].unique():
    plt.figure(figsize=(10, 5))
    subset = grouped[grouped['année'] == year]
    colors = ['red' if x == 1 else 'blue' for x in subset['depasse_seuil']]
    months = [calendar.month_name[m] for m in subset['mois']]
    plt.bar(months, subset['valeur'], color=colors)
    plt.axhline(y=OBJECTIF_QUALITE_PM10_MENSUEL, color='green', linestyle='--', linewidth=2, label=f'Seuil {OBJECTIF_QUALITE_PM10_MENSUEL}')
    plt.xlabel('Mois')
    plt.ylabel('Valeur moyenne mensuelle PM10')
    plt.title(f'Valeurs moyennes mensuelles PM10 pour l\'année {year}')
    plt.xticks(rotation=45)
    plt.show()


## Ajout d'alerte quand les variables de seuils sont dépassées

### Pour les valeurs de seuils annuels

In [ ]:
# Convertir les colonnes de date en format datetime si ce n'est pas déjà fait
dfPM10['date_heure_tu'] = pd.to_datetime(dfPM10['date_heure_tu'])

# Grouper les données par année et département
dfPM10['année'] = dfPM10['date_heure_tu'].dt.year
grouped = dfPM10.groupby(['année', 'departement_nom'])[['valeur']].mean().reset_index()

# Arrondir les valeurs moyennes à un chiffre après la virgule
grouped['valeur'] = grouped['valeur'].round(1)

# Créer la colonne indiquant si la moyenne annuelle dépasse un des seuils
grouped['depasse_objectif_qualite'] = grouped['valeur'] > OBJECTIF_QUALITE_PM10
grouped['depasse_valeur_limite'] = grouped['valeur'] > VALEUR_LIMITE_ANNUELLE_PM10

# Regrouper les alertes par type
alertes_qualite = grouped[grouped['depasse_objectif_qualite']]
alertes_limite = grouped[grouped['depasse_valeur_limite']]

# Afficher les alertes par type pour les moyennes annuelles avec le département
if not alertes_qualite.empty:
    print("Alertes pour l'Objectif de Qualité annuel:")
    for index, row in alertes_qualite.iterrows():
        print(f"Alerte: Valeur moyenne annuelle {row['valeur']} dépasse l'Objectif de Qualité annuel ({OBJECTIF_QUALITE_PM10} µg/m3) en {row['année']} dans le département {row['departement_nom']}")
else:
    print("Aucune alerte pour l'Objectif de Qualité annuel.")

if not alertes_limite.empty:
    print("\nAlertes pour la Valeur Limite annuelle:")
    for index, row in alertes_limite.iterrows():
        print(f"Alerte: Valeur moyenne annuelle {row['valeur']} dépasse la Valeur Limite annuelle ({VALEUR_LIMITE_ANNUELLE_PM10} µg/m3) en {row['année']} dans le département {row['departement_nom']}")
else:
    print("Aucune alerte pour la Valeur Limite annuelle.")



### Pour les valeurs de seuils mensuels

In [ ]:
import pandas as pd
import calendar

# Convertir les colonnes de date en format datetime si ce n'est pas déjà fait
dfPM10['date_heure_tu'] = pd.to_datetime(dfPM10['date_heure_tu'])

# Grouper les données par année, mois et département
dfPM10['année'] = dfPM10['date_heure_tu'].dt.year
dfPM10['mois'] = dfPM10['date_heure_tu'].dt.month
grouped = dfPM10.groupby(['année', 'mois', 'departement_nom'])[['valeur']].mean().reset_index()

# Arrondir les valeurs moyennes à un chiffre après la virgule
grouped['valeur'] = grouped['valeur'].round(1)

# Créer la colonne indiquant si la moyenne mensuelle dépasse le seuil
grouped['depasse_objectif_qualite'] = grouped['valeur'] > OBJECTIF_QUALITE_PM10_MENSUEL

# Extraire les mois en lettres
grouped['mois_lettres'] = grouped['mois'].apply(lambda x: calendar.month_name[x])

# Regrouper les alertes par type
alertes_qualite = grouped[grouped['depasse_objectif_qualite']]

# Afficher les alertes par type pour les moyennes mensuelles avec le département
if not alertes_qualite.empty:
    print("Alertes pour l'Objectif de Qualité mensuel:")
    for index, row in alertes_qualite.iterrows():
        print(f"Alerte: Valeur moyenne {row['valeur']} dépasse l'Objectif de Qualité mensuel ({OBJECTIF_QUALITE_PM10_MENSUEL} µg/m3) en {row['mois_lettres']} {row['année']} dans le département {row['departement_nom']}")
else:
    print("Aucune alerte pour l'Objectif de Qualité mensuel.")
